In [ ]:
import pandas as pd
from dotenv import load_dotenv
from INGESTION.captioning import convert_image_to_base64, get_fish_description_from_watsonxai

In [2]:
df = pd.read_csv("app/fish-description-files/Marine_Fish_Species_Full_Description_test.csv")
df

,Fish Name,Summary Description,Image Links
0,Brownbanded bamboo shark,This small carpet sharkis commonly found insha...,https://object-storage.example.com/brownbanded...
1,Whale shark,The world's largest livingshark and fish speci...,https://object-storage.example.com/whale_shark...
2,Zebra shark,A distinctive carpetshark with fivelongitudina...,https://object-storage.example.com/zebra_shark...
3,Coral catshark,A small bottom-dwellingshark found in coral re...,https://object-storage.example.com/coral_catsh...
4,Blacktip reef shark,A medium-sized reef shark easily identified by...,https://object-storage.example.com/blacktip_re...
...,...,...,...
86,Moorish idol,An iconic reef fishwith distinctiveblack and w...,https://object-storage.example.com/moorish_ido...
87,Little dragonfish,"A small, dragon-like fish withwing-like pector...",https://object-storage.example.com/little_drag...
88,Indian mackerel,"A small, schooling fishimportant in commercial...",https://object-storage.example.com/indian_mack...
89,Trumpetfish,An elongated fish with a tube-like snout used ...,https://object-storage.example.com/trumpetfish...


In [7]:
from app.retrieve import get_fish_description_from_watsonxai,convert_image_to_base64

In [20]:
import os
import glob
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

# Define the root directory
root_dir = 'app/fish_images'

# Recursively find all .png files
image_paths = glob.glob(os.path.join(root_dir, '**', '*.png'), recursive=True)

df_physical_description = pd.DataFrame()
# Show each image
i= 0
rang_img = 3
for image_path in image_paths:
    print(f"Displaying: {image_path}")
    img_sting = convert_image_to_base64(image_path)
    df_physical_description.iloc[i,'image_path'] = [image_path]
    df_physical_description.iloc[i,'image_base64'] = [img_sting]
    df_physical_description.iloc[i,'fish_description'] = get_fish_description_from_watsonxai(img_sting)
    i+=3
    if i== 10:
        break
    # plt.imshow(img)
    # plt.title(os.path.basename(image_path))
    # plt.axis('off')  # Hide axis
    # plt.show()
df_physical_description

Displaying: app/fish_images/Bluefin-trevally/bluefin-trevally-001.png


IndexError: iloc cannot enlarge its target object

In [18]:
df_physical_description

,image_path,image_base64,fish_description
0,app/fish_images/White-spotted-boxfish/white-sp...,iVBORw0KGgoAAAANSUhEUgAAA+gAAAKZCAIAAACkyJZjAA...,"{\n 'body': 'The fish has a rounded, puffy ..."
